In [ ]:
"""
**IGNORE**

First we need to import some stuff. Feel free to ignore this cell.

If you're interested, each import has an associated comment that explains why
the import is useful/necessary.
"""

# Needed for various filesystem tasks (os.path.exists etc.)
import os
# Used for checking how many cores are available for processing.
import multiprocessing
# Used for constructing paths.
from pathlib import Path

# Essential for all mathematical operations we'll be carrying out.
import numpy as np

# diffraction_utils is a library developed at Diamond by Richard Brearton
# (richard.brearton@diamond.ac.uk) to ease the task of parsing data files and
# carrying out some common calculations. Here, we'll be using it to define
# frames of reference, and parse nexus files.
# We also use diffraction_utils' Region object to specify regions of interest/
# background regions.
from diffraction_utils import Frame, Region

# The following imports are required for the core of the calculation code, also
# written by Richard Brearton (richard.brearton@diamond.ac.uk).
# This is the central Experiment object, which stores all the logic related to
# mapping the experiment.
from fast_rsm.experiment import Experiment


In [ ]:
"""
**ESSENTIAL**

This cell requires action! Make sure you set all of the variables defined here.
"""

# What was your scattering geometry/how was your sample mounted? Options are
# 'horizontal', 'vertical' and 'DCD'.
setup = 'vertical'

# Set local_data_path if your data isn't stored on the diamond system any more
# (for example if it's on a memory stick or scratch drive).
local_data_path = None
# Set this if you want to save the output somewhere other than the processing
# folder. Be warned, this could take up a lot of space.
local_output_path = None

# If you're processing on the cluster, you need to populate the next few fields.
# The experiment number, used to work out where your data is stored.
experiment_number = 'si31429-1'

# The sub-directory containing your experimental data. Leave as None if unused.
# Otherwise, if the data was stored in a subdirectory called "day_1", e.g.
#   /dls/i07/data/2022/si32333-1/day_1/
# then you should use:
#   data_sub_directory = "day_1"
data_sub_directory = "sample1/"

# The year the experiment took place.
year = 2022

# The scan numbers of the scans that we want to use to produce this reciprocal
# space map. For example, the default value of scan_numbers shows how to specify
# every scan between number 421772 and 421778 inclusive, but skipping scan
# number 421776.
scan_numbers = [446509]

# Uncomment the following to set scan_numbers equal to every scan number between
# scan_start and scan_stop (inclusive of scan_stop):
# scan_start = 439168
# scan_stop = 439176
# scan_numbers = list(range(scan_start, scan_stop + 1))

# The beam centre, as can be read out from GDA, in pixel_x, pixel_y. If your
# map looks wacky, you probably cocked this up.
beam_centre = (243, 92)

# The distance between the sample and the detector (or, if using the DCD, the
# distance between the receiving slit and the detector). Units of meters.
detector_distance = 930e-3

# The frame/coordinate system you want the map to be carried out in.
# Options for frame_name argument are:
#     Frame.hkl     (map into hkl space - requires UB matrix in nexus file)
#     Frame.sample_holder   (standard map into 1/Å)
#     Frame.lab     (map into frame attached to lab.)
#     Frame.qpar_qperp (as in Frame.lab, but with no component along beam)
#
# Please note that the q_parallel q_perpendicular frame is a bad choice. Q is a
# three dimensional vector. By choosing this frame, you are consenting to
# throwing one of these dimensions in the bin. Instead consider Frame.lab.
#
# Options for coordinates argument are:
#     Frame.cartesian   (normal cartesian coords: hkl, Qx Qy Qz, etc.)
#     Frame.polar       (cylindrical polar with cylinder axis along l/Qz)
#
# Frame.polar will give an output like a more general version of PyFAI.
# Frame.cartesian is for hkl maps and Qx/Qy/Qz. Any combination of frame_name
# and coordinates will work, so try them out; get a feel for them.
frame_name = Frame.hkl
coordinates = Frame.cartesian

# How large would you like your output file to be, in MB? 100MB normally gives
# very good resolution without sacrificing performance. If you want something
# higher resolution, feel free, but be aware that the performance of the map and
# the analysis will start to suffer above around 1GB.
# Max file size is 2GB (2000MB).
output_file_size = 50

# This is for loading into binoculars. If set to false, .npy and .vtr files
# will be saved, for manual analysis and paraview, respectively.
save_binoculars_h5 = True

# Are you using the DPS system?
using_dps = False

# The DPS central pixel locations are not typically recorded in the nexus file.
# UNITS OF METERS, PLEASE (everything is S.I., except energy in eV).
dpsx_central_pixel = 0
dpsy_central_pixel = 0
dpsz_central_pixel = 0

# Note: THESE CAN BE HAPPILY AUTO CALCULATED.
# These take the form:
# volume_start = [h_start, k_start, l_start]
# volume_stop = [h_stop, k_stop, l_stop]
# volume_step = [h_step, k_step, l_step]
# Leave as None if you don't want to specify them. You can specify whichever
# you like (e.g. you can specify step and allow start/stop to be auto
# calculated)
volume_start = None
volume_stop = None
volume_step = None

# Only use this if you need to load your data from a .dat file.
load_from_dat = False

In [ ]:
"""
**MASKING**

This cell contains details on how to mask pixels. You can either mask a series
of individual pixels, mask rectangular regions of pixels, or dynamically mask
pixels based on their intensity (not recommended).
"""

# If you have a small number of hot pixels to mask, specify them one at a time
# in a list. In other words, it should look like:
# specific_pixels = [(pixel_x1, pixel_y1), (pixel_x2, pixel_y2)]
# Or, an exact example, where we want to mask pixel (233, 83) and pixel 
# (234, 83), where pixel coordinates are (x, y):
# 
# specific_pixels = [
#     (233, 83),
#     (234, 83)
# ]
# 
# Leave specific pixels as None if you dont want to mask any specific pixels.
# For this dataset we need to mask pixel (x=233, y=83)
specific_pixels = None

# If you want to specify an entire region of pixels to mask, do so here.
# This is done using a "Region" object. To make a Region, give it start_x, 
# stop_x, start_y, start_y, as follows:
# 
mask_1 = Region(0, 180, 0, 30)
mask_2 = Region(320, -1, 165, -1)
mask_3 = Region(0, -1, 0, 30)
mask_4 = Region(0, -1, 165, -1)
 
# Where my_mask_region runs in x from pixel 3 to 6 inclusive, and runs in y from
# pixel 84 to 120 inclusive. You can make as many mask regions as you like, just
# make sure that you put them in the mask_regions list, as follows:
# mask_regions = [my_mask_region, Region(1, 2, 3, 4)]
# 
# If you don't want to use any mask regions, just leave mask_regions equal to
# None.
mask_regions = None

# Ignore pixels with an intensity below this value. If you don't want to ignore
# any pixels, then set min_intensity = None. This is useful for dynamically
# creating masks (which is really useful for generating masks from -ve numbers).
min_intensity = 0.


In [ ]:
"""
**IGNORE**
This cell prepares the calculation. You probably shouldn't change anything here
unless you know what you're doing.
"""

# Which synchrotron axis should become the out-of-plane (001) direction.
# Defaults to 'y'; can be 'x', 'y' or 'z'.
if setup == 'vertical':
    oop = 'x'
elif setup == 'horizontal':
    oop = 'y'
elif setup == 'DCD':
    oop = 'y'
else:
    raise ValueError(
        "Setup not recognised. Must be 'vertical', 'horizontal' or 'DCD.")

if output_file_size > 2000:
    raise ValueError("output_file_size must not exceed 2000. "
                     f"Value received was {output_file_size}.")

# Max number of cores available for processing.
num_threads = multiprocessing.cpu_count()

# Work out where the data is.
if local_data_path is None:
    data_dir = Path(f"/dls/i07/data/{year}/{experiment_number}/")
else:
    data_dir = Path(local_data_path)
# data_dir = Path(f"/Users/richard/Data/i07/{experiment_number}/")

# Store this for later.
if local_output_path is None:
    processing_dir = data_dir / "processing"
else:
    processing_dir = Path(local_output_path)
if data_sub_directory is not None:
    data_dir /= Path(data_sub_directory)

# Here we calculate a sensible file name that hasn't been taken.
i = 0
save_file_name = f"mapped_scan_{scan_numbers[0]}_{i}"
save_path = processing_dir / save_file_name
# Make sure that this name hasn't been used in the past.
while (os.path.exists(str(save_path) + ".npy") or
       os.path.exists(str(save_path) + ".vtk") or
       os.path.exists(str(save_path) + "_l.txt") or
       os.path.exists(str(save_path) + "_tth.txt") or
       os.path.exists(str(save_path) + "_Q.txt") or
       os.path.exists(save_path)):
    i += 1
    save_file_name = f"mapped_scan_{scan_numbers[0]}_{i}"
    save_path = processing_dir / save_file_name

    if i > 1e7:
        raise ValueError(
            "Either you tried to save this file 10000000 times, or something "
            "went wrong. I'm going with the latter, but exiting out anyway.")


# Work out the paths to each of the nexus files. Store as pathlib.Path objects.
nxs_paths = [data_dir / f"i07-{x}.nxs" for x in scan_numbers]

# Construct the Frame object from the user's preferred frame/coords.
map_frame = Frame(frame_name=frame_name, coordinates=coordinates)

# Prepare the pixel mask. First, deal with any specific pixels that we have.
# Note that these are defined (x, y) and we need (y, x) which are the
# (slow, fast) axes. So: first we need to deal with that!
if specific_pixels is not None:
    specific_pixels = specific_pixels[1], specific_pixels[0]

# Now deal with any regions that may have been defined.
# First make sure we have a list of regions.
if isinstance(mask_regions, Region):
    mask_regions = [mask_regions]

# Now swap (x, y) for each of the regions.
if mask_regions is not None:
    for region in mask_regions:
        region.x_start, region.y_start = region.y_start, region.x_start
        region.x_end, region.y_end = region.y_end, region.x_end

# Finally, instantiate the Experiment object.
experiment = Experiment.from_i07_nxs(
    nxs_paths, beam_centre, detector_distance, setup, 
    using_dps=using_dps)

experiment.mask_pixels(specific_pixels)
experiment.mask_regions(mask_regions)

In [ ]:
"""
**POTENTIALLY REQUIRED**

This cell is for changing metadata that is stored in, or inferred from, the
nexus file. This is generally for more nonstandard stuff.
"""

for i, scan in enumerate(experiment.scans):
    # Deal with the dps offsets.
    if scan.metadata.data_file.using_dps:
        scan.metadata.data_file.dpsx -= dpsx_central_pixel
        scan.metadata.data_file.dpsy -= dpsy_central_pixel
        scan.metadata.data_file.dpsz -= dpsz_central_pixel

    # Load from .dat files if we've been asked.
    if load_from_dat:
        dat_path = data_dir / f"{scan_numbers[i]}.dat"
        scan.metadata.data_file.populate_data_from_dat(dat_path)

    # This is where you might want to overwrite some data that was recorded
    # badly in the nexus file. See (commented out) examples below.
    # scan.metadata.data_file.probe_energy = 12500
    # scan.metadata.data_file.transmission = 0.4
    # scan.metadata.data_file.using_dps = True
    # scan.metadata.data_file.ub_matrix = np.array([
    #     [1, 0, 0],
    #     [0, 1, 0],
    #     [0, 0, 1]
    # ])

    # Would you like to skip any images in any scans? Do so here!
    # This shows how to skip the 9th in the 3rd scan (note the zero counting).
    # if i == 2:
    #     scan.skip_images.append(8)




In [ ]:
"""
**IGNORE**

This cell contains all of the logic for running the calculation. You shouldn't
run this on your local computer, it'll either raise an exception or take
forever.
"""
from fast_rsm.diamond_utils import save_binoculars_hdf5


if __name__ == "__main__":
    # Calculate and save a binned reciprocal space map, if requested.
    experiment.binned_reciprocal_space_map(
        num_threads, map_frame, output_file_size=output_file_size, oop=oop,
        min_intensity_mask=min_intensity,
        output_file_name=save_path, 
        volume_start=volume_start, volume_stop=volume_stop,
        volume_step=volume_step)

    if save_binoculars_h5:
        save_binoculars_hdf5(str(save_path) + ".npy", str(save_path) + '.hdf5')
        print(f"\nSaved BINoculars file to {save_path}.hdf5.\n")

    # Finally, print that it's finished We'll use this to work out when the
    # processing is done.
    print("PROCESSING FINISHED.")

class DontContinue(Exception):
    """Raise to stop processing on the cluster at this cell"""

raise DontContinue("Processing complete!!\n"
                   "This is intentionally raised to stop the processing. "
                   "Never worry about the presence of this 'error'.")


In [ ]:
"""
**ESSENTIAL**

This is the cell that you should execute to run this notebook on the cluster.

DO NOT EXECUTE THIS MULTIPLE TIMES. IT WILL SUBMIT MULTIPLE JOBS TO THE CLUSTER.
PLEASE BE RESPONSIBLE.
"""
# We need this to grab the current working directory.
import os

# We'll need this to run the program that will submit the cluster job.
# This module isn't needed for the calculation itself, which is why it is
# imported here.
import subprocess

# First, we save this as "map.py". Make sure it doesn't already exist.
try:
    os.remove("map.py")
except OSError:
    pass

# Convert this notebook to a python script in our home directory.
!jupyter nbconvert --to script map.ipynb


# Submit the job, which in turn loads the Hamilton module and runs:
# qsub -pe smp 40 -l m_mem_free=2.5G -P i07 cluster_job.sh
subprocess.run(
    ["sh", "/dls_sw/apps/fast_rsm/current/scripts/cluster_sub.sh"])



In [ ]:
"""
The following cells contain convenience tools for e.g. interacting with and
visualising data.
"""


In [ ]:
"""
**RUN THIS**

You should run this cell, but don't worry about its contents.

This cell just defines a few handy functions that will be used below.
"""

import os
import numpy as np

def most_recent_cluster_output():
    """
    Returns the filename of the most recent cluster stdout output.
    """
    # Get all the cluster job files that have been created.
    files = [x for x in os.listdir() if x.startswith("cluster_job.sh.o")]
    numbers = [int(x[16:]) for x in files]

    # Work out which cluster job is the most recent.
    most_recent_job_no = np.max(numbers)
    most_recent_file = ""
    for file in files:
        if str(most_recent_job_no) in file:
            most_recent_file = file

    return most_recent_file


def most_recent_cluster_error():
    """
    Returns the filename of the most recent cluster stderr output.
    """
    # Get all the cluster job files that have been created.
    files = [x for x in os.listdir() if x.startswith("cluster_job.sh.e")]
    numbers = [int(x[16:]) for x in files]

    # Work out which cluster job is the most recent.
    most_recent_job_no = np.max(numbers)
    most_recent_file = ""
    for file in files:
        if str(most_recent_job_no) in file:
            most_recent_file = file

    return most_recent_file


In [ ]:
"""
Are we nearly there yet?

Executing this cell tells you if your most recent cluster submission has
finished executing.
"""

most_recent_file = most_recent_cluster_output()

# Open that file, and see if it ends in "Finished."
with open(most_recent_file, 'r') as f:
    lines = f.read().splitlines()
    
    # Check if there's nothing in the file yet.
    if len(lines) == 0:
        print("Processing either not started or crashed.")
    last_line = lines[-1].strip()
    if last_line.startswith("PROCESSING FINISHED."):
        print("Most recent processing has completed.")
    else:
        print(
            "Processing job has started on the cluster, but isn't finished. "
            "It could have crashed, or it could still be running.")
    


In [ ]:
"""
This cell contains functions that you can use to read the most recent cluster
output and error logs.
"""

def print_recent_output():
    """Prints the most recent cluster output log file."""
    with open(most_recent_cluster_output(), 'r') as f:
        lines = f.read().splitlines()
        for line in lines:
            print(line)

def print_recent_error():
    """Prints the most recent cluster error log file."""
    with open(most_recent_cluster_error(), 'r') as f:
        lines = f.read().splitlines()
        for line in lines:
            print(line)

print("Cluster stderr (error logs) below:")
print_recent_error()
print("\n\n\nCluster stdout (output logs) below:")
print_recent_output()

In [ ]:
"""
This cell computes projections on your data (e.g. maps it to I vs Q, I vs tth,
I vs L, qparalel_qperpendicular*). Intensity is logged, but just comment out
the line of code that logs it to plot linear (ctrl+f np.log).

If you want structure factor, replace np.log with np.sqrt.

*coming soon TM
"""

import numpy as np

import plotly.express as px

from diffraction_utils import I07Nexus
from fast_rsm.binning import weighted_bin_1d
from fast_rsm.diamond_utils import get_volume_and_bounds, q_to_theta


# This is the path to the .npy file containing your data.
path_to_npy = "/path/to/file.npy"

# Please only set one of these to be True, or something will probably go wrong.
map_to_q = True
map_to_tth = False
map_to_l = False

# If you want to map to tth, please specify the energy of the beam (in eV).
# If not, don't worry about this value because it wont be used.
beam_energy = None

# Alternatively, to grab the beam energy from a nexus file, use this code.
# path_to_nx = "/path/to/nx"
# beam_energy = I07Nexus(path_to_nx).probe_energy

# If you want to set bin_size, then set number_of_bins = None.
# If you set both, bin_size will win.
number_of_bins = 1000
bin_size = None

# BELOW ARE JUST DETAILS OF THE CALCULATION.

# Grab the volume, and the bounds on the volume.
volume, start, stop, step = get_volume_and_bounds(path_to_npy)

# Calculate Q everywhere.
q_x = np.arange(start[0], stop[0], step[0], dtype=np.float32)
q_y = np.arange(start[1], stop[1], step[1], dtype=np.float32)
q_z = np.arange(start[2], stop[2], step[2], dtype=np.float32)
if map_to_l:
    q_x *= 0
    q_y *= 0
q_x, q_y, q_z = np.meshgrid(q_x, q_y, q_z, indexing='ij')

# Now we can work out the |Q| for each voxel.
q_x_squared = np.square(q_x)
q_y_squared = np.square(q_y)
q_z_squared = np.square(q_z)
q_lengths = np.sqrt(q_x_squared + q_y_squared + q_z_squared)

# It doesn't make sense to keep the 3D shape because we're about to map
# to a 1D space anyway.
volume = volume.ravel()
q_lengths = q_lengths.ravel()

# If we're mapping to tth, map q->tth now.
if map_to_tth:
    q_lengths = 2*q_to_theta(q_lengths, beam_energy)

# If we haven't been given a bin_size, we need to calculate it.
min_q = float(np.min(q_lengths))
max_q = float(np.max(q_lengths))

if bin_size is None:
    # Work out the bin_size from the range of |Q| values.
    bin_size = (max_q - min_q)/number_of_bins

# Now that we know the bin_size, we can make an array of the bin edges.
bins = np.arange(min_q, max_q, bin_size)

# Use this to make intensity & count arrays of the correct size.
intensity = np.zeros_like(bins, dtype=np.float32)
count = np.zeros_like(intensity, dtype=np.uint32)

# Do the binning. This can take a long time to execute, potentially, since we're
# binning 100 million elements or so (depending on your volume size).
intensity = weighted_bin_1d(
    q_lengths, volume, intensity, count, min_q, max_q, bin_size)

# Normalise by the counts.
intensity /= count.astype(np.float32)

# Take the log.
intensity = np.log(intensity)

if map_to_q:
    labels = {'x': "Q(1/Å)", 'y': 'ln(Intensity) (a.u.)'}
if map_to_l:
    labels = {'x': "l (unitless)", 'y': 'ln(Intensity) (a.u.)'}
if map_to_tth:
    labels = {'x': "2θ (degrees)", 'y': 'ln(Intensity) (a.u.)'}
# Finally, do the plot!
fig = px.line(x=bins, y=intensity, labels=labels)
fig.show()

In [ ]:
"""
This cell is for working out background regions for background subtraction
along l. You can do this in binoculars, though.
"""

import plotly.express as px

from fast_rsm.diamond_utils import get_volume_and_bounds, q_to_theta

# This is the path to the .npy file containing your data.
path_to_npy = "/path/to/file.npy"
l_value = 1
# Options are 'log', 'linear' and 'sqrt'
plot_mode = 'log'

# Grab the volume, and the bounds on the volume.
volume, start, stop, step = get_volume_and_bounds(path_to_npy)

# Replace NaN with 0.
volume = np.nan_to_num(volume)

def l_to_arr_coord(l_value: float, start, step):
    """
    Convenience function for converting between an l value and an array
    coordinate. Useful for for h, k slices at a specific value of l.
    """
    return int((l_value - start[2])/step[2])

l_coordinate = l_to_arr_coord(l_value, start, step)

image = volume[:, :, l_coordinate]

if plot_mode not in ['log', 'linear', 'sqrt']:
    raise ValueError(
        f"plot_mode must be one of 'log', 'linear' or sqrt'. Got {plot_mode}.")

if plot_mode == 'log':
    image = np.log(image)
if plot_mode == 'sqrt':
    image = np.sqrt(image)

fig = px.imshow(image)
fig.show()


In [ ]:
"""
This cell is for carrying out background subtraction along l. You should run
the previous cell first to take a look at (and load) your volume.
"""

from diffraction_utils import Region

# These regions shouldn't overlap, or you'll subtract the pixels in the overlap
# regions multiple times.
signal_region = Region(1, 20, 15, 40)
bkg_region_1 = Region(1, 20, 15, 40)
bkg_region_2 = Region(1, 20, 15, 40)
bkg_regions = [bkg_region_1, bkg_region_2]

for l in range(volume.shape[-1]):
    # Make a slice along this value of l.
    l_slice = volume[:, :, l]

    # Calculate the weighted average of the noise in the background regions.
    means = [np.mean(l_slice[x.slice]) for x in bkg_regions]
    num_pixels = [x.num_pixels for x in bkg_regions]
    weighted_mean = np.sum(
        [means[i] * x for i, x in enumerate(num_pixels)])/np.sum(num_pixels)

    # Subtract this from the l slice.
    l_slice -= weighted_mean
